In [243]:
pip install pandas sqlalchemy mysqlclient

In [244]:
# import pacakge to make http request
import requests

# import json library to decode json
import json

# import for data manipulation and analysis
import pandas as pd

# import to interact with mysql
from sqlalchemy import create_engine

## Get event details

In [251]:
# API key
from google.colab import userdata
api_key = userdata.get('TICKETMASTER_API_KEY')

# API endpoint for searching events
url = 'https://app.ticketmaster.com/discovery/v2/events.json'

# Parameters for the API call
params = {
        'apikey': api_key,
        'classificationName': 'music',
        'dmaId': '324', # Code for LA
        'size': 5 # Number of events to retrieve
    }

# Make the request
event_response = requests.get(url, params=params)



SyntaxError: invalid syntax (<ipython-input-251-cae3ec76e36a>, line 2)

In [249]:
# Check status of request
event_response

<Response [200]>

In [250]:
# View text contents of the response
event_response.text

'{"_embedded":{"events":[{"name":"West Conf Qtrs: Suns at Timberwolves Rd 1 Hm Gm 3","type":"event","id":"Z7r9jZ1A7FvqE","test":false,"url":"https://www.ticketmaster.com/event/Z7r9jZ1A7FvqE","locale":"en-us","images":[{"ratio":"16_9","url":"https://s1.ticketm.net/dam/a/c70/98044e71-9085-471b-9f21-78525c745c70_1340061_RECOMENDATION_16_9.jpg","width":100,"height":56,"fallback":false},{"ratio":"3_2","url":"https://s1.ticketm.net/dam/a/c70/98044e71-9085-471b-9f21-78525c745c70_1340061_RETINA_PORTRAIT_3_2.jpg","width":640,"height":427,"fallback":false},{"ratio":"16_9","url":"https://s1.ticketm.net/dam/a/c70/98044e71-9085-471b-9f21-78525c745c70_1340061_RETINA_PORTRAIT_16_9.jpg","width":640,"height":360,"fallback":false},{"ratio":"16_9","url":"https://s1.ticketm.net/dam/a/c70/98044e71-9085-471b-9f21-78525c745c70_1340061_RETINA_LANDSCAPE_16_9.jpg","width":1136,"height":639,"fallback":false},{"ratio":"3_2","url":"https://s1.ticketm.net/dam/a/c70/98044e71-9085-471b-9f21-78525c745c70_1340061_TABLE

In [229]:
#Format into Json
json.loads(event_response.text)

{'_embedded': {'events': [{'name': 'Imagine Dragons: LOOM WORLD TOUR',
    'type': 'event',
    'id': 'vvG1iZbah89wNY',
    'test': False,
    'url': 'https://www.ticketmaster.com/imagine-dragons-loom-world-tour-hollywood-california-10-23-2024/event/0B006096D05F28F0',
    'locale': 'en-us',
    'images': [{'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dam/a/e8c/5265b9d2-a06c-4dc8-a432-a8dd9d042e8c_SOURCE',
      'width': 2426,
      'height': 1365,
      'fallback': False},
     {'ratio': '3_2',
      'url': 'https://s1.ticketm.net/dam/a/e8c/5265b9d2-a06c-4dc8-a432-a8dd9d042e8c_ARTIST_PAGE_3_2.jpg',
      'width': 305,
      'height': 203,
      'fallback': False},
     {'ratio': '4_3',
      'url': 'https://s1.ticketm.net/dam/a/e8c/5265b9d2-a06c-4dc8-a432-a8dd9d042e8c_CUSTOM.jpg',
      'width': 305,
      'height': 225,
      'fallback': False},
     {'ratio': '3_2',
      'url': 'https://s1.ticketm.net/dam/a/e8c/5265b9d2-a06c-4dc8-a432-a8dd9d042e8c_RETINA_PORTRAIT_3_2.jpg',


In [230]:
# Assign variable to formated reponse
event_response_json = json.loads(event_response.text)

In [231]:
# extract event details from a JSON response
def extract_event_details(event_response_json):
    events = event_response_json['_embedded']['events']
    all_event_details = []

    for event in events:
        # Formatting event details
        event_details = f"Event: {event['name']}, Date and Time: {event['dates']['start']['localDate']} at {event['dates']['start']['localTime']} local time, Location: {event['_embedded']['venues'][0]['name']}"

        # Formatting venue details
        venue = event['_embedded']['venues'][0]
        venue_details = f"Venue: {venue['name']}, Address: {venue['address']['line1']}, {venue['city']['name']}, {venue['state']['stateCode']} {venue['postalCode']}"

        all_event_details.append((event_details, venue_details))

    return all_event_details


# Extract and print details for up to 5 events
all_event_details = extract_event_details(event_response_json)

for event_details, venue_details in all_event_details:
    print(event_details)
    print(venue_details)
    print("\n")  # Add a newline for better readability between events


Event: Imagine Dragons: LOOM WORLD TOUR, Date and Time: 2024-10-23 at 19:00:00 local time, Location: Hollywood Bowl
Venue: Hollywood Bowl, Address: 2301 N Highland Ave, Hollywood, CA 90068


Event: Imagine Dragons: LOOM WORLD TOUR, Date and Time: 2024-10-22 at 19:00:00 local time, Location: Hollywood Bowl
Venue: Hollywood Bowl, Address: 2301 N Highland Ave, Hollywood, CA 90068


Event: P!NK: Summer Carnival 2024, Date and Time: 2024-09-15 at 18:30:00 local time, Location: Dodger Stadium
Venue: Dodger Stadium, Address: 1000 Elysian Park Ave, Los Angeles, CA 90012


Event: Fool in Love, Date and Time: 2024-08-31 at 11:00:00 local time, Location: Hollywood Park Grounds
Venue: Hollywood Park Grounds, Address: 1011 Stadium Drive, Inglewood, CA 90305


Event: Aventura, Date and Time: 2024-05-04 at 20:00:00 local time, Location: Crypto.com Arena
Venue: Crypto.com Arena, Address: 1111 S. Figueroa St., Los Angeles, CA 90017




In [232]:
# Create Dataframe containing concert information
def extract_event_details_to_dataframe(event_response_json):
    events = event_response_json['_embedded']['events']
    data = []

    for event in events:
        # Collecting event and venue details
        event_details = {
            'Name': event['name'],
            'Date': event['dates']['start']['localDate'],
            'Time': event['dates']['start']['localTime'],
            'Event_Location': event['_embedded']['venues'][0]['name'],
            'Venue_Name': event['_embedded']['venues'][0]['name'],
            'Address': event['_embedded']['venues'][0]['address']['line1'],
            'City': event['_embedded']['venues'][0]['city']['name'],
            'State': event['_embedded']['venues'][0]['state']['stateCode'],
            'Postal_Code': event['_embedded']['venues'][0]['postalCode']
        }
        data.append(event_details)

    # Creating a DataFrame
    df = pd.DataFrame(data)
    return df

# Assuming event_response_json is your JSON data from the API
df_events = extract_event_details_to_dataframe(event_response_json)
print(df_events)


                               Name        Date      Time  \
0  Imagine Dragons: LOOM WORLD TOUR  2024-10-23  19:00:00   
1  Imagine Dragons: LOOM WORLD TOUR  2024-10-22  19:00:00   
2        P!NK: Summer Carnival 2024  2024-09-15  18:30:00   
3                      Fool in Love  2024-08-31  11:00:00   
4                          Aventura  2024-05-04  20:00:00   

           Event_Location              Venue_Name                Address  \
0          Hollywood Bowl          Hollywood Bowl    2301 N Highland Ave   
1          Hollywood Bowl          Hollywood Bowl    2301 N Highland Ave   
2          Dodger Stadium          Dodger Stadium  1000 Elysian Park Ave   
3  Hollywood Park Grounds  Hollywood Park Grounds     1011 Stadium Drive   
4        Crypto.com Arena        Crypto.com Arena   1111 S. Figueroa St.   

          City State Postal_Code  
0    Hollywood    CA       90068  
1    Hollywood    CA       90068  
2  Los Angeles    CA       90012  
3    Inglewood    CA       90305  
4

In [233]:
# view data frame contents
df_events

,Name,Date,Time,Event_Location,Venue_Name,Address,City,State,Postal_Code
0,Imagine Dragons: LOOM WORLD TOUR,2024-10-23,19:00:00,Hollywood Bowl,Hollywood Bowl,2301 N Highland Ave,Hollywood,CA,90068
1,Imagine Dragons: LOOM WORLD TOUR,2024-10-22,19:00:00,Hollywood Bowl,Hollywood Bowl,2301 N Highland Ave,Hollywood,CA,90068
2,P!NK: Summer Carnival 2024,2024-09-15,18:30:00,Dodger Stadium,Dodger Stadium,1000 Elysian Park Ave,Los Angeles,CA,90012
3,Fool in Love,2024-08-31,11:00:00,Hollywood Park Grounds,Hollywood Park Grounds,1011 Stadium Drive,Inglewood,CA,90305
4,Aventura,2024-05-04,20:00:00,Crypto.com Arena,Crypto.com Arena,1111 S. Figueroa St.,Los Angeles,CA,90017


In [235]:
# Create a mapping of artist names to their IDs
Artist_ID = {
    'Imagine Dragons': 1435919,
    'P!NK': 718655,
    'Aventura': 973009
}

In [236]:
# Function to extract the main artist's name from the event name and map it to the artist ID
def map_artist_id(event_name):
    for artist in artist_ids.keys():
        if artist.lower() in event_name.lower():
            return artist_ids[artist]
    return None

# Apply the function to the DataFrame
df_events['Artist_ID'] = df_events['Name'].apply(map_artist_id)

In [237]:
# view updated data frame
df_events

,Name,Date,Time,Event_Location,Venue_Name,Address,City,State,Postal_Code,Artist_ID
0,Imagine Dragons: LOOM WORLD TOUR,2024-10-23,19:00:00,Hollywood Bowl,Hollywood Bowl,2301 N Highland Ave,Hollywood,CA,90068,1435919.0
1,Imagine Dragons: LOOM WORLD TOUR,2024-10-22,19:00:00,Hollywood Bowl,Hollywood Bowl,2301 N Highland Ave,Hollywood,CA,90068,1435919.0
2,P!NK: Summer Carnival 2024,2024-09-15,18:30:00,Dodger Stadium,Dodger Stadium,1000 Elysian Park Ave,Los Angeles,CA,90012,718655.0
3,Fool in Love,2024-08-31,11:00:00,Hollywood Park Grounds,Hollywood Park Grounds,1011 Stadium Drive,Inglewood,CA,90305,NaN
4,Aventura,2024-05-04,20:00:00,Crypto.com Arena,Crypto.com Arena,1111 S. Figueroa St.,Los Angeles,CA,90017,973009.0


In [238]:
# save df to a csv file
df_events.to_csv('yelp_tacos_90045.csv', index=False)

In [239]:
# creates data base engine
def create_db_engine(user, password, host, port, database):
    return create_engine(f'mysql+mysqldb://{user}:{password}@{host}:{port}/{database}')


In [240]:
# Function to load CSV into DataFrame
def load_csv(file_path):
    return pd.read_csv(file_path)

# Function to upload DataFrame to MySQL
def upload_to_mysql(df, engine, table_name):
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False, chunksize=500)

# Main script execution
if __name__ == "__main__":
    # Database credentials and connection details
    USER = 'admin'
    PASSWORD = 'isba_4715'
    HOST = 'isba-dev-01.ctpnruf7xrki.us-east-1.rds.amazonaws.com'
    PORT = '3306'
    DATABASE = 'sql_project'
    TABLE_NAME = 'Events'  # Make sure this table is already created in your MySQL database

    # Path to the CSV file
    CSV_FILE_PATH = '/content/event_details.csv'

    # Create a database engine
    engine = create_db_engine(USER, PASSWORD, HOST, PORT, DATABASE)

    # Load data from CSV
    dataframe = load_csv(CSV_FILE_PATH)

    # Upload data to MySQL
    upload_to_mysql(dataframe, engine, TABLE_NAME)
    print("CSV data has been uploaded successfully to MySQL!")

CSV data has been uploaded successfully to MySQL!
